In [2]:
import requests

In [3]:
import re

In [4]:
import csv

In [5]:
from bs4 import BeautifulSoup

In [6]:
urllist = list()

In [7]:
r = requests.get("http://www.zinch.cn/ss/us",params = {"page":"1"})

In [8]:
soup = BeautifulSoup(r.text,"html.parser")

In [9]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html dir="ltr" lang="zh-hans" xml:lang="zh-hans" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Zinch中国 - 美国院校搜索, 美国院校库 | Zinch</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/sites/all/themes/zinchus/images/iphone-icon.png" rel="apple-touch-icon-precomposed"/>
<meta content="留学,美国留学信息,DIY留学,免中介留学,美国大学信息,美国留学免费咨询." name="keywords"/>
<meta content="Zinch中国(zinch.cn)拥有国内最完善的美国大学及其他海外院校的信息库，包括录取要求，录取率，校园介绍，留学前途等详尽数据和分析." name="description"/>
<link href="/sites/all/themes/zinchus/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/sites/all/themes/zinchus/css/forms.css" media="all" rel="stylesheet" type="te

In [49]:
soup = BeautifulSoup(r.text,"html.parser")

In [60]:
c =  list();
for tag in soup.find_all("div"):
    if "class" in list(tag.attrs.keys()):
        if "school-ranking-search-link" in tag['class']:
            c.append(tag.text[9:].strip())
            

In [64]:
ranklist = list()
incre = 1
with open("rank.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(1,22):
        params = {"page":i}
        r = requests.get("http://www.zinch.cn/ss/us",params = params)
        soup = BeautifulSoup(r.text,"html.parser")
        for tag in soup.find_all("div"):
            if "class" in list(tag.attrs.keys()):
                if "school-ranking-search-link" in tag['class']:
                    ranklist.append(tag.text[9:].strip())
        for tag in soup.find_all("h2"):
            if "class" in list(tag.attrs.keys()):
                if "properties-name-en" in tag['class']:
                    urllist.append(tag.findNext().attrs['href']);
                    if tag.findNext().text.find('(') == -1:
                        spamwriter.writerow([incre,tag.findNext().text,ranklist[incre-1]])
                    else:
                        ind = tag.findNext().text.find('(')
                        spamwriter.writerow([incre,tag.findNext().text[0:ind-1],ranklist[incre-1]])
                    incre = incre+1
                        

In [11]:
urllist

['/princeton-university',
 '/harvard-university',
 '/universityofchicago',
 '/yale-university',
 '/stanford-university',
 '/columbia-university',
 '/massachusetts-institute-technology',
 '/pennsylvania-university',
 '/duke-university',
 '/johns-hopkins-university',
 '/dartmouth-college',
 '/california-institute-of-technology-caltech',
 '/northwestern-university',
 '/brown-university',
 '/cornell-university',
 '/notre-dame-university',
 '/vanderbilt-university',
 '/rice-university',
 '/washington-university-in-st-louis',
 '/uc-berkeley',
 '/emory-university',
 '/georgetown-university',
 '/southern-california-university',
 '/ucla',
 '/virginia-university',
 '/cmu',
 '/tufts-university',
 '/michigan-university',
 '/wake-forest-university',
 '/north-carolina-university-chapel-hill',
 '/boston-college',
 '/university-of-rochester',
 '/william-mary-the-college',
 '/georgia-institute-technology',
 '/brandeis-university',
 '/new-york-university',
 '/uc-santa-barbara',
 '/cwru',
 '/uc-irvine',


In [30]:
logolist = list()

In [31]:
from urllib.request import urlretrieve

In [32]:
schoolurl = list();

In [33]:
for i in range(1,22):
    params = {"page":i}
    r = requests.get("http://www.zinch.cn/ss/us",params = params)
    soup = BeautifulSoup(r.text,"html.parser")
    for tag in soup.find_all("img"):
        if "class" in list(tag.attrs.keys()):
            if "imagecache-75x75" in tag['class']:
                logolist.append(tag.attrs['src']);

In [36]:
for i in range(0,len(urllist)):
    schoolurl.append(urllist[i][1:])

In [42]:
for i in range(0,len(schoolurl)):
    rank.iloc[i,1] = schoolurl[i];
    rank.loc[i,"usrank"] = i+1;

In [43]:
rank.columns = ["id","image","usrank"]

In [47]:
rank.T.to_json("image.json")

In [39]:
## Crawling the school logo image
for i in range(0,len(urllist)):
    fileurl = "./schoollogo/"+urllist[i][1:]+".jpg"
    urlretrieve(logolist[i],fileurl);

In [88]:
r = requests.get("http://www.zinch.cn/princeton-university/major_grad")

In [139]:
with open("zinchinfo.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(0,210):
        r=requests.get("http://www.zinch.cn"+urllist[i]+"/major_grad")
        if r.status_code == 200:
            soup = BeautifulSoup(r.text,"html.parser")
            ans = list();
            for tag in soup.find_all("tr"):
                ans.append(tag.findNext().findNext().text);
            spamwriter.writerow(ans);
        else:
            spamwriter.writerow([np.nan])
            

In [20]:
import pandas as pd

In [21]:
import numpy as np

In [294]:
info = pd.read_csv("./zinchinfo.csv",header=None)

In [295]:
info = info.iloc[:,0:5]

In [296]:
for i in range(0,210):
    toe = info.iloc[i,0]
    sat = info.iloc[i,1]
    rate = info.iloc[i,4]/info.iloc[i,3]
    if str(toe) == "nan":
        continue;
    elif re.search("[0-9]+",toe)!=None:
        info.iloc[i,0] = re.search("[0-9]+",toe).group(0)
    else:
        info.iloc[i,0] = np.nan
    if str(sat) == "nan":
        continue;
    elif re.search("[0-9]+ - [0-9]+",sat)!=None:
        info.iloc[i,1] = re.search("[0-9]+ - [0-9]+",sat).group(0)
    else:
        info.iloc[i,1] = np.nan
    info.iloc[i,3] = rate;
info = info.iloc[:,0:4]

In [299]:
info.to_json("zinchdata.json")

In [ ]:
##0:toefl, 1:sat, 2:accrate, 3:interate

In [40]:
rank = pd.read_csv("rank.csv",header=None)

In [41]:
rank

,0,1
0,1,Princeton University
1,2,Harvard University
2,3,The University of Chicago
3,4,Yale University
4,5,Stanford University
5,6,Columbia University in the City of New York
6,7,Massachusetts Institute of Technology
7,8,University of Pennsylvania
8,9,Duke University
9,10,Johns Hopkins University


In [26]:
rank.T.to_json("rank.json")

In [25]:
rank.columns = ["id","school"]

In [331]:
info.columns = ["toefl","sat","acptrate","interate"]

In [332]:
info.T.to_json("zinchdata.json")

In [330]:
info

,0,1,2,3
0,105,2120 - 2380,7%,0.118542
1,100,1800 - 2400,6%,0.299074
2,100,2140 - 2390,6%,0.176739
3,105,2040 - 2350,5%,0.099174
4,100,2120 - 2350,9%,0.150000
5,NaN,2100 - 2350,7%,0.123987
6,90,2090 - 2350,6%,0.088118
7,NaN,2090 - 2330,11%,0.065650
8,100,2050 - 2370,10%,0.013339
9,105,2160 - 2390,9%,0.102747
